# **Trajectory** on the reconstructed lane

**Trajectory computation**

In [26]:
import cv2
import numpy as np
from pathlib import Path
import csv

Loading files paths

In [27]:
VIDEO_NUMBER = "7"
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Recording_{VIDEO_NUMBER}.mp4")
CSV_FILE_PATH = str(PROJECT_ROOT / "data" / "auxiliary_data" / "reconstructed_positions" / f"Transformed_positions_processed_{VIDEO_NUMBER}.csv")
OUTPUT_VIDEO_PATH = str(PROJECT_ROOT / "data" / f"recording_{VIDEO_NUMBER}" / f"Reconstructed_trajectory_processed_{VIDEO_NUMBER}.mp4")

Creating video *Tracked_output.mp4*

In [28]:
# Create image
width = 106    # Approximate pixels for 1.0668m
height = 1829  # Approximate pixels for 18.29m
brown_color = (135, 184, 222)  # RGB for burly wood
alley = np.full((height, width, 3), brown_color, dtype=np.uint8)

# Load video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
    
# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

In [29]:
# Read circle positions from CSV
positions = {}
with open(CSV_FILE_PATH, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        if len(row) >= 3:
            frame_str, x_str, y_str = row[:3]
            if frame_str.strip() and x_str.strip() and y_str.strip():
                try:
                    frame_num = int(frame_str)
                    x = int(float(x_str))  # scale down to fit image
                    y = int(float(y_str))
                    positions[frame_num] = (x, y)
                except ValueError:
                    continue  # skip rows with non-numeric values

In [30]:
frame_count = 0
trajectory = []

while cap.isOpened():
    ret, _ = cap.read()
    if not ret:
        break

    frame = alley.copy()

    # Check if this frame has a position
    if frame_count in positions:
        x, y = positions[frame_count]
        trajectory.append((x, y))
        #print(f"Frame {frame_count}: Ball at ({x}, {y})")

    # Draw trajectory path
    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i - 1], trajectory[i], (0, 0, 255), 2)

    # Draw current ball position
    if frame_count in positions:
        cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)  # red dot

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
print(f"Tracking video saved to {OUTPUT_VIDEO_PATH}")

Tracking video saved to /home/davic/projects/IACV_project/bowling-analysis/data/recording_7/Reconstructed_trajectory_processed_7.mp4


IMPROVEMENTS: Make is smoothy, more visible and maybe use another recostructed lane to print che trajectory